In [4]:
import urllib.parse
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import pandas as pd
import time
from bs4 import BeautifulSoup
import re

In [5]:
def configure_driver():
    chrome_options = Options()
    # options.add_argument('--headless')  # Menjalankan Chrome tanpa antarmuka grafis
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36")
    chrome_options.add_argument('--disable-dev-shm-usage')  # Mencegah masalah memori
    chrome_options.add_argument('--disable-gpu')  # Opsi tambahan untuk meningkatkan stabilitas
    chrome_options.add_argument('--remote-debugging-port=9222')

    # Tentukan path ChromeDriver
    service = Service(r'chromedriver-win64\chromedriver.exe')
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

In [6]:
df_company_not_found = pd.read_excel("company2_not_found.xlsx")

In [7]:
# Function to search Bing for a company by name
def search_bing(company_name):
    driver = configure_driver()
    website_url = "N/A"

    try:
        # Use Bing search URL
        driver.get(f"https://www.bing.com/search?q={company_name}")
        time.sleep(2)

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "h2 > a"))
        )

        first_result = driver.find_element(By.CSS_SELECTOR, "h2 > a")
        website_url = first_result.get_attribute("href")

    except Exception as e:
        print(f"Failed to find result for {company_name} on Bing: {e}")
    finally:
        driver.quit()
        return website_url

In [8]:
# Update the dataframe by searching for websites using Bing
for idx, row in df_company_not_found.iterrows():
    company_name = row['Nama Perusahaan']
    
    # Check if 'Situs Resmi' is still 'Data Belum Ditemukan'
    if row['Situs Resmi'] == "Data Belum Ditemukan":
        print(f"Searching website for {company_name}")
        
        # Call the search_bing function to get the website URL
        website_url = search_bing(company_name)
        
        # If a website is found, update the 'Situs Resmi' column
        if website_url != "N/A":
            df_company_not_found.at[idx, 'Situs Resmi'] = website_url
            print(f"Found website for {company_name}: {website_url}")

Searching website for Asuransi Bina Dana Arta Tbk
Found website for Asuransi Bina Dana Arta Tbk: https://myoona.id/en/about-us/
Searching website for PT Aspirasi Hidup Indonesia Tbk
Found website for PT Aspirasi Hidup Indonesia Tbk: https://ahi.id/id
Searching website for PT Samcro Hyosung Adilestari Tbk.
Found website for PT Samcro Hyosung Adilestari Tbk.: https://samcro.co.id/
Searching website for PT Adhi Commuter Properti Tbk
Found website for PT Adhi Commuter Properti Tbk: https://www.adcp.co.id/
Searching website for PT Anugerah Spareparts Sejahtera Tbk.
Found website for PT Anugerah Spareparts Sejahtera Tbk.: https://aegisfilter.co.id/
Searching website for PT Asia Sejahtera Mina Tbk
Found website for PT Asia Sejahtera Mina Tbk: http://asiamina.com/
Searching website for Asuransi Harta Aman Pratama Tbk
Found website for Asuransi Harta Aman Pratama Tbk: https://asuransi-harta.co.id/
Searching website for PT FKS Food Sejahtera Tbk
Found website for PT FKS Food Sejahtera Tbk: https

In [9]:
df_company_not_found

,Kode Perusahaan,Nama Perusahaan,Link,Tanggal Pencatatan,Alamat Kantor,Alamat Email,Telepon,Fax,NPWP,Situs,...,Papan Pencatatan,Bidang Usaha Utama,Sektor,Subsektor,Industri,Subindustri,Biro Administrasi Efek,Sekretaris Nama,Sekretaris Telepon,Situs Resmi
0,ABDA,Asuransi Bina Dana Arta Tbk,https://www.idx.co.id/id/perusahaan-tercatat/p...,06 Jul 1989,Plaza Asia Lt. 27 Jl. Jend. Sudirman Kav. 59 J...,fausia.tanoso@oona-insurance.co.id,5140 1688,5140 1698,01.362.014.1-054.000,www.myoona.id,...,Pemantauan Khusus,Asuransi Umum,Keuangan,Asuransi,Asuransi,Asuransi Umum,PT. Raya Saham Registra,Fausia Isti Tanoso,021-51401688,https://myoona.id/en/about-us/
1,ACES,PT Aspirasi Hidup Indonesia Tbk,https://www.idx.co.id/id/perusahaan-tercatat/p...,06 Nov 2007,"Gedung Kawan Lama, Jl. Puri Kencana No. 1, Kem...",corsec@kawanlamacorp.com,021 - 5828282,021 - 5824022,01.721.123.6-054.000,https://ahi.id/,...,Utama,Perdagangan,Barang Konsumen Non-Primer,Perdagangan Ritel,Ritel Khusus,Ritel Barang Rumah Tangga,PT. EDI Indonesia,Gregory Sugyono Widjaja,+62215828282,https://ahi.id/id
2,ACRO,PT Samcro Hyosung Adilestari Tbk.,https://www.idx.co.id/id/perusahaan-tercatat/p...,11 Jan 2024,"Jl. Raya Serang KM 17,2 Desa Bojong Kecamatan ...",corsec@samcro.co.id,021-5960543,021-5960548,01.068.028.8-052.000,https//www.samcro.co.id,...,Pengembangan,Industri dan perdagangan Hook dan Loop,Barang Konsumen Non-Primer,Pakaian & Barang Mewah,Pakaian & Barang Mewah,Tekstil,PT. Adimitra Jasa Korpora,Endri Kusumah S.T,021-5960543,https://samcro.co.id/
3,ADCP,PT Adhi Commuter Properti Tbk,https://www.idx.co.id/id/perusahaan-tercatat/p...,21 Mei 2021,"Jl. Pengantin Ali No. 88, Ciracas\n\nKota Jaka...",corsec@adcp.co.id,(021) 228 229 80,(021) 228 220 81,85.227.029.7-093.000,www.adcp.co.id,...,Pengembangan,Perhotelan dan Real Estate,Properti & Real Estat,Properti & Real Estat,Pengelola & Pengembang Real Estat,Pengembang & Operator Real Estat,-,Bayu Purwana,021 228 229 80,https://www.adcp.co.id/
4,AEGS,PT Anugerah Spareparts Sejahtera Tbk.,https://www.idx.co.id/id/perusahaan-tercatat/p...,11 Sep 2023,"Jalan Soekarno Hatta No. 134 Losari Kidul, Los...",corsec@aegisfilter.co.id,+62231831035,NaN,0739 4091 3442 6000,https://www.aegisfilter.co.id/,...,Pemantauan Khusus,Bergerak dalam bidang usaha Industri Filter Ke...,Barang Konsumen Non-Primer,Otomotif & Komponen Otomotif,Komponen Otomotif,Suku Cadang Otomotif,PT. Bima Registra,Jeihan Sumawi Putra,081911401699,https://aegisfilter.co.id/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,ZATA,PT Bersama Zatta Jaya Tbk,https://www.idx.co.id/id/perusahaan-tercatat/p...,10 Nov 2022,"Komplek Industri Prapanca No.24, Cigondewah Ka...",irvan@elcorps.com,(022) 86017900,NaN,31.569.863.9-428.000,www.elcorps.com,...,Pemantauan Khusus,Aktivitas Perusahaan Holding dan aktivitas Usa...,Barang Konsumen Non-Primer,Perdagangan Ritel,Ritel Khusus,Ritel Pakaian & Tekstil,PT. Adimitra Jasa Korpora,Irvan Rachmawan,+62 82115558881,https://elcorps.co.id/
515,ZBRA,PT Dosni Roha Indonesia Tbk,https://www.idx.co.id/id/perusahaan-tercatat/p...,01 Agt 1991,"Graha Dosni Roha, Jl Budi Raya No. 9B, Kemangg...",ida.widayani@dnr.id,021-530 5600,-,01.451.869.0-054.000,www.dnr.id,...,Pengembangan,Perusahaan Holding dan Angkutan Bermotor untuk...,Perindustrian,Perusahaan Holding Multi Sektor,Perusahaan Holding Multi-sektor,Perusahaan Holding Multi-sektor,PT BSR Indonesia,Ida Widayani,021-27883900,https://www.bing.com/alink/link?url=http%3a%2f...
516,ZINC,PT Kapuas Prima Coal Tbk,https://www.idx.co.id/id/perusahaan-tercatat/p...,16 Okt 2017,"Jl. Pantai Indah Selatan I, Elang Laut Blok A ...",corsec@ptkpc.com,021 - 29676236,021 - 29676234,02.386.972.0-091.000,www.ptkpc.com,...,Pemantauan Khusus,Pertambangan dan Perdagangan,Barang Baku,Barang Baku,Logam & Mineral,Logam & Mineral Lainnya,-,Lucky Tajo,021 29676236,https://kapuasprima.co.id/
517,ZONE,PT Mega Perintis Tbk.,https://www.idx.co.id/id/perusahaan-tercatat/p...,12 De

In [10]:
# Save the updated dataframe to a new CSV
df_company_not_found.to_csv('updated_company_data.csv', index=False)
print("Data has been updated and saved to 'updated_company_data.csv'")

Data has been updated and saved to 'updated_company_data.csv'
